# Compare EC profit redistribution

This notebook aims at comparing the profit redistribution for an Energy Community
using the package EnergyCommunity.jl and TheoryOfGames.jl

## Configuration

### Paths

In [1]:
input_file = joinpath(@__DIR__, "../data/energy_community_model.yml")  # Input file

output_file_isolated = joinpath(@__DIR__, "../results/output_file_NC.xlsx")  # Output file - model users alone
output_plot_isolated = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_NC.png")  # Output png file of plot - model users alone

output_file_combined = joinpath(@__DIR__, "../results/output_file_EC.xlsx")  # Output file - model Energy community
output_plot_combined = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_EC.pdf")  # Output png file of plot - model energy community

output_plot_sankey_agg = joinpath(@__DIR__, "../results/Img/sankey_EC.png")  # Output plot of the sankey plot related to the aggregator case
output_plot_sankey_noagg = joinpath(@__DIR__, "../results/Img/sankey_NC.png")  # Output plot of the sankey plot related to the no aggregator case

enum_mode_file = "enum_mode_datasest.jld2"  # file used to store the enumerative results
total_results_file = "total_results_file.jld2"  # file to store all major results

overwrite_files = true  # when true, output files are overwritten

true

### Import libraries

In [2]:
using Revise
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using TheoryOfGames
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify

### Solver configurations

In [3]:
# General optimizer
OPTIMIZER = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0, "Threads"=>10)

# Optimizer for row-generation techniques, used in the IterMode of TheoryOfGames.jl
OPTIMIZER_ROW_GENERATION = optimizer_with_attributes(Gurobi.Optimizer,
    "OutputFlag"=>1,
    "LogToConsole"=>0,
    "MIPGap"=>0.1,
    # "MIPFocus"=>1,
    "TimeLimit"=>1000,
    "LogFile"=>"gurobi.log",
    "Threads"=>10,
    # "NoRelHeurTime"=>10,
    "PoolSolutions"=>20,
    "PoolSearchMode"=>1,
)

MathOptInterface.OptimizerWithAttributes(Gurobi.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("OutputFlag") => 1, MathOptInterface.RawOptimizerAttribute("LogToConsole") => 0, MathOptInterface.RawOptimizerAttribute("MIPGap") => 0.1, MathOptInterface.RawOptimizerAttribute("TimeLimit") => 1000, MathOptInterface.RawOptimizerAttribute("LogFile") => "gurobi.log", MathOptInterface.RawOptimizerAttribute("Threads") => 10, MathOptInterface.RawOptimizerAttribute("PoolSolutions") => 20, MathOptInterface.RawOptimizerAttribute("PoolSearchMode") => 1])

### Energy Community options

In [4]:
NO_AGG_GROUP = GroupANC();  # type of aggregation when the Aggregator does not belong to the coalition.
                            # options: GroupANC() or GroupNC()
BASE_GROUP = GroupNC();     # base type of aggregation (it shall be GroupNC)

## General

In [5]:
ENV["COLUMNS"] = 1000  # to print more columns in the output

1000

## Optimize the Energy Community in the COperative configuration

In [6]:
# Read data from excel file
ECModel = ModelEC(input_file, EnergyCommunity.GroupCO(), OPTIMIZER)

# Reset the user set to use all stored users (10)
# reset_user_set!(ECModel)
set_user_set!(ECModel, ["user$id" for id=1:8])

# Build the model
build_model!(ECModel)

# Optimize the model
optimize!(ECModel)

Academic license - for non-commercial use only - expires 2023-02-04


An Energy Community Model
Energy Community problem for a Cooperative Model
User set: ["user1", "user2", "user3", "user4", "user5", "user6", "user7", "user8"]


Solved model

## Calculate the reward distribution functions by enumerative techniques (EnumMode in TheoryOfGames.jl)

### Create enumerative mode

Calculation of the enumerative mode used to compute several reward distribution functions

In [7]:
tick()
enum_mode = EnumMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP)
time_elapsed_enum=tok()
println("EnumMode calculated with elapsed time [min]: $(time_elapsed_enum/60)")

┌ Info:  started timer at: 2022-06-07T17:05:01.329
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04


0.0%┣                                             ┫ 0/511 [00:00<00:-26, -0s/it]


0.2%┣                                          ┫ 1/511 [00:00<Inf:Inf, InfGs/it]


0.4%┣▏                                              ┫ 2/511 [00:01<05:57, 1it/s]


...

99.4%┣███████████████████████████████████████████▊┫ 508/511 [19:29<00:07, 2s/it]

Academic license - for non-commercial use only - expires 2023-02-04



99.6%┣███████████████████████████████████████████▉┫ 509/511 [19:37<00:05, 2s/it]


Academic license - for non-commercial use only - expires 2023-02-04
EnumMode calculated with elapsed time [min]: 20.08973759166667


100.0%┣███████████████████████████████████████████┫ 511/511 [19:50<00:00, 2s/it]
100.0%┣███████████████████████████████████████████┫ 511/511 [19:50<00:00, 2s/it]


Save enum mode

In [8]:
if !isfile(total_results_file) || overwrite_files
    save(enum_mode_file, enum_mode)
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\Davide\.julia\packages\JLD2\k9Gt0\src\JLD2.jl:233


Load EnumMode (when needed)

In [9]:
# enum_mode = load(enum_mode_file, EnumMode())

### Calculate reward distribution using EnumMode

In [10]:
tick()
shapley_dist_enum = shapley_value(enum_mode)  # shapley value
time_elapsed_shapley_enum=tok()

tick()
nucleolus_dist_enum = nucleolus(enum_mode, OPTIMIZER)  # nucleolus
time_elapsed_nucleolus_enum=tok()

tick()
varcore_dist_enum = var_in_core(enum_mode, OPTIMIZER)  # variance in core
time_elapsed_varcore_enum=tok()

tick()
varleastcore_dist_enum, val_minsurplus_enum, model_dist_enum = var_least_core(
    enum_mode, OPTIMIZER; raw_outputs=true
)  # variance least core (include raw outputs for comparison purposes)
time_elapsed_varleastcore_enum=tok();

┌ Info:  started timer at: 2022-06-07T17:25:20.268
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


Academic license - for non-commercial use only - expires 2023-02-04


┌ Info:  started timer at: 2022-06-07T17:25:20.619
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54
0.4%┣▏                                              ┫ 2/511 [00:02<13:29, 2s/it]
100.0%┣█████████████████████████████████████████┫ 511/511 [00:02<00:00, 307it/s]


┌ Info:  started timer at: 2022-06-07T17:25:23.957
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


Academic license - for non-commercial use only - expires 2023-02-04


┌ Info:  started timer at: 2022-06-07T17:25:27.111
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


Academic license - for non-commercial use only - expires 2023-02-04


Store the EnumMode reward distribution into a DataFrame for ease their use

In [11]:
# vector of the users
user_set_agg = [EC_CODE; get_user_set(ECModel)]

"Auxiliary function to order the output of reward distributions and return them as vectors"
vectorize_rewards(reward_dist, users_list=user_set_agg) = [reward_dist[u] for u in users_list]

# dataframe of reward distributions for the enumerative mode
df_reward_enum = DataFrame(
    user_set=user_set_agg,
    shapley_enum=vectorize_rewards(shapley_dist_enum),
    nucleolus_enum=vectorize_rewards(nucleolus_dist_enum),
    varcore_enum=vectorize_rewards(varcore_dist_enum),
    varleastcore_enum=vectorize_rewards(varleastcore_dist_enum),
)

# dataframe of the time requirements
dict_time_enum = Dict(
    "EnumMode"=>time_elapsed_enum,
    "shapley_enum"=>time_elapsed_shapley_enum+time_elapsed_enum,
    "nucleolus_enum"=>time_elapsed_nucleolus_enum+time_elapsed_enum,
    "varcore_enum"=>time_elapsed_varcore_enum+time_elapsed_enum,
    "varleastcore_enum"=>time_elapsed_varleastcore_enum+time_elapsed_enum,
)

df_reward_enum

,user_set,shapley_enum,nucleolus_enum,varcore_enum,varleastcore_enum
,String,Float64,Float64,Float64,Float64
1,EC,33645.8,45456.5,37437.9,40554.4
2,user1,5304.78,3093.52,5297.5,3093.52
3,user2,8693.99,4022.1,7115.62,4360.17
4,user3,11926.3,6879.99,9196.55,7218.06
5,user4,7012.39,7587.29,9948.34,7925.36
6,user5,41512.2,47816.0,37437.9,45176.3
7,user6,9387.19,3546.45,6301.9,3546.45
8,user7,30005.7,35652.3,37437.9,40554.4
9,user8,11264.2,4698.59,8579.04,6324.08


## Calculate the reward distribution functions by row-generation techniques (IterMode in TheoryOfGames.jl)

### Define the row-generation decomposition mode by IterMode

In [12]:
iter_mode = IterMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP, optimizer=OPTIMIZER_ROW_GENERATION)

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04


Model not optimized
Solved model


IterMode(["EC", "user1", "user2", "user3", "user4", "user5", "user6", "user7", "user8"], EnergyCommunity.var"#utility_callback_by_subgroup#664"{EnergyCommunity.var"#objective_callback_by_subgroup#585"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#613"{ModelEC}}, EnergyCommunity.var"#objective_callback_by_subgroup#490"{JuMP.Containers.DenseAxisArray{Float64, 1, Tuple{Vector{String}}, Tuple{JuMP.Containers._AxisLookup{Dict{String, Int64}}}}}}(EnergyCommunity.var"#objective_callback_by_subgroup#585"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#613"{ModelEC}}(An Energy Community Model
Energy Community problem for a Cooperative Model
User set: ["user1", "user2", "user3", "user4", "user5", "user6", "user7", "user8"]
, EnergyCommunity.var"#objective_callback_by_subgroup#613"{ModelEC}(An Energy Community Model
Energy Community problem for a Aggregating-Non-Cooperative Model
User set: ["user1", "user2", "user3", "user4", "user5", "user6", "user7", "user8"]
)), Ene

Model not optimized

Define coalitions to precompute when performing the iterative procedure

In [13]:
# include all coalitions having no more than preload_max_size users
preload_max_size = 2

preload_coalitions = Iterators.flatten([combinations([EC_CODE; ECModel.user_set], k) for k = 1:preload_max_size])

Base.Iterators.Flatten{Vector{Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{String}}}}}(Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{String}}}[Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{String}}}(Combinatorics.var"#reorder#10"{Vector{String}}(["EC", "user1", "user2", "user3", "user4", "user5", "user6", "user7", "user8"]), Combinatorics.Combinations(9, 1)), Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{String}}}(Combinatorics.var"#reorder#10"{Vector{String}}(["EC", "user1", "user2", "user3", "user4", "user5", "user6", "user7", "user8"]), Combinatorics.Combinations(9, 2))])

### Calculate reward distribution using row-generation technique

Variance Least Core using IterMode

In [14]:
tick()
varleastcore_dist_iter, min_surplus_varlc_iter, history_varlc_iter, model_dist_varlc_iter = var_least_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
)
time_elapsed_varleastcore_iter=tok()
println("Variance Least Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varleastcore_iter/60)")

┌ Info:  started timer at: 2022-06-07T17:26:09.843
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


PHASE 1: Start first least core analysis



Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Preload 45 coalitions


Iteration       Upper bound     Current value   Tol. [%]        Benefit distribution


1                1.50e+04       -3.94e+04        137.99         [14953.877050342338, 14953.877050342338, 14953.877050342338, 16622.44289514976, 14953.877050342338, 25740.051050740527, 16746.198752985052, 24874.627651595045, 14953.877050342338]


...


13               2.76e+03       -7.21e+03        138.20         [38204.30840146856, 3093.5164971039194, 14328.811051432345, 7218.063926423334, 7925.361559331624, 38204.30840146867, 3546.450037671253, 38204.30840146866, 8027.577325813925]


14               2.76e+03       -2.14e+02        107.77         [41527.18905822708, 3093.5164971040736, 4360.169081156152, 7218.06392642318, 7925.361559332292, 41527.18905822713, 3546.450037671103, 41527.189058227195, 8027.577325813875]


15               2.76e+03        1.05e+03        61.82          [40042.415408691864, 3093.5164971040676, 4360.169081156391, 7218.063926424108, 7925.361559331884, 44496.73635729711, 3546.4500376711076, 40042.41540869171, 8027.577325814214]


16               2.76e+03        2.08e+03        24.66          [40894.162056931935, 3093.5164971037952, 4360.169081157319, 7218.063926424398, 7925.361559333381, 44496.73635729663, 3546.4500376713804, 40894.16205693202, 6324.084029332742]


17               2.76e+03        2.76e+03        0.00           [40554.40474664577, 3093.5164971036515, 4360.169081156094, 7218.063926423408, 7925.361559332137, 45176.2509778712, 3546.4500376715223, 40554.40474664579, 6324.084029332604]
PHASE 2: Completed second phase

Variance Least Core - IterMode calculated with elapsed time [min]: 74.48898629


Variance Core method using IterMode

In [15]:
tick()
varcore_dist_iter, min_surplus_varcore_iter, history_varcore_iter, model_dist_varcore_iter = var_in_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
)
time_elapsed_varcore_iter=tok()
println("Variance Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varcore_iter/60)")

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Preload 45 coalitions
Iteration       Upper bound     Current value   Tol. [%]        Benefit allocation


┌ Info:  started timer at: 2022-06-07T18:40:39.696
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


1               -4.93e+04        0.00e+00        100.00         [17639.189511353565, 17639.18951135357, 17639.18951135357, 17639.18951135357, 17639.18951135357, 17639.18951135357, 17639.18951135357, 17639.18951135357, 17639.18951135357]


2               -2.39e+04        0.00e+00        100.00         [29971.26412540051, 7773.529820115999, 7773.529820116001, 7773.529820116, 7773.529820116002, 29971.264125400514, 7773.529820116001, 29971.264125400518, 29971.264125400514]


3               -6.20e+03        0.00e+00        100.00         [35336.17221668917, 2.4500948641420324e-12, 9716.912275144998, 9716.91227514531, 9716.912275145187, 35336.17221668882, 9716.912275145129, 35336.17221668875, 13876.539851534937]

...

10              -2.47e+02        0.00e+00        100.00         [37390.18251892066, 5443.26249538844, 7104.081826350564, 9205.543614139764, 10094.096687108287, 37390.18251892069, 6301.89606628694, 37390.18251892061, 8433.277356146116]


11              -2.35e+02        0.00e+00        100.00         [37390.182518918984, 5196.6634847912765, 7350.6808369439805, 9205.54361414752, 9847.497676516725, 37390.182518922426, 6301.8960662853715, 37390.1825189203, 8679.876366738468]


12              -1.43e+02        0.00e+00        100.00         [37390.18251892089, 5196.66348480597, 7115.615109772135, 9440.609341300069, 9847.497676525758, 37390.18251892081, 6301.896066286334, 37390.18251892071, 8679.876366728642]


13              -4.24e+01        0.00e+00        100.00         [37437.92168559815, 5339.880984838324, 7115.615109773704, 9154.174341233724, 9990.715176558226, 37437.92168559802, 6301.896066287069, 37437.921685598354, 8536.658866696618]


14              -2.51e-09        0.00e+00        0.25           [37437.92168560002, 5297.504034741061, 7115.615109771428, 9196.551291332922, 9948.33822645868, 37437.921685599125, 6301.896066284523, 37437.92168560066, 8579.035816792302]
Variance Core - IterMode calculated with elapsed time [min]: 55.414629795


Store results as a DataFrame

In [32]:
# dataframe of reward distributions for the enumerative mode
df_reward_iter = DataFrame(
    user_set=user_set_agg,
    varcore_iter=vectorize_rewards(varcore_dist_iter),
    varleastcore_iter=vectorize_rewards(varleastcore_dist_iter),
)

# dataframe of the time requirements
dict_time_iter = Dict(
    "IterMode"=>0.0,
    "varcore_iter"=>time_elapsed_varcore_iter,
    "varleastcore_iter"=>time_elapsed_varleastcore_iter,
)

df_reward_iter

,user_set,varcore_iter,varleastcore_iter
,String,Float64,Float64
1,EC,37437.9,40554.4
2,user1,5297.5,3093.52
3,user2,7115.62,4360.17
4,user3,9196.55,7218.06
5,user4,9948.34,7925.36
6,user5,37437.9,45176.3
7,user6,6301.9,3546.45
8,user7,37437.9,40554.4
9,user8,8579.04,6324.08


## Group all results and save them

Merge results

In [33]:
df_reward = innerjoin(df_reward_enum, df_reward_iter, on=:user_set)
df_reward

,user_set,shapley_enum,nucleolus_enum,varcore_enum,varleastcore_enum,varcore_iter,varleastcore_iter
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,EC,33645.8,45456.5,37437.9,40554.4,37437.9,40554.4
2,user1,5304.78,3093.52,5297.5,3093.52,5297.5,3093.52
3,user2,8693.99,4022.1,7115.62,4360.17,7115.62,4360.17
4,user3,11926.3,6879.99,9196.55,7218.06,9196.55,7218.06
5,user4,7012.39,7587.29,9948.34,7925.36,9948.34,7925.36
6,user5,41512.2,47816.0,37437.9,45176.3,37437.9,45176.3
7,user6,9387.19,3546.45,6301.9,3546.45,6301.9,3546.45
8,user7,30005.7,35652.3,37437.9,40554.4,37437.9,40554.4
9,user8,11264.2,4698.59,8579.04,6324.08,8579.04,6324.08


In [34]:
dict_time = merge(dict_time_enum, dict_time_iter)
dict_time

Dict{String, Float64} with 8 entries:
  "EnumMode"          => 1205.38
  "varcore_iter"      => 3324.88
  "varleastcore_enum" => 1207.17
  "shapley_enum"      => 1205.74
  "IterMode"          => 0.0
  "varleastcore_iter" => 4469.34
  "nucleolus_enum"    => 1208.72
  "varcore_enum"      => 1208.54

Save results

In [35]:
if !isfile(total_results_file) || overwrite_files
    jldsave(total_results_file; df_reward, dict_time)
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\Davide\.julia\packages\JLD2\k9Gt0\src\JLD2.jl:233


In [36]:
# print to latex the reward table
set_default(fmt = "%.2f", convert_unicode = false)

df_reward_mod = copy(df_reward)
df_reward_mod[!, 2:end] = df_reward[!, 2:end] ./ 1000  # change € unit to k€
tex_df_reward = latexify(df_reward_mod; env=:table, latex=false)

L"\begin{tabular}{ccccccc}
user_set & shapley_enum & nucleolus_enum & varcore_enum & varleastcore_enum & varcore_iter & varleastcore_iter\\
EC & 33.65 & 45.46 & 37.44 & 40.55 & 37.44 & 40.55\\
user1 & 5.30 & 3.09 & 5.30 & 3.09 & 5.30 & 3.09\\
user2 & 8.69 & 4.02 & 7.12 & 4.36 & 7.12 & 4.36\\
user3 & 11.93 & 6.88 & 9.20 & 7.22 & 9.20 & 7.22\\
user4 & 7.01 & 7.59 & 9.95 & 7.93 & 9.95 & 7.93\\
user5 & 41.51 & 47.82 & 37.44 & 45.18 & 37.44 & 45.18\\
user6 & 9.39 & 3.55 & 6.30 & 3.55 & 6.30 & 3.55\\
user7 & 30.01 & 35.65 & 37.44 & 40.55 & 37.44 & 40.55\\
user8 & 11.26 & 4.70 & 8.58 & 6.32 & 8.58 & 6.32\\
\end{tabular}
"

In [44]:
names(df_reward)[2:end]

6-element Vector{String}:
 "shapley_enum"
 "nucleolus_enum"
 "varcore_enum"
 "varleastcore_enum"
 "varcore_iter"
 "varleastcore_iter"

In [49]:
df_computational_time = DataFrame(dict_time)[!, names(df_reward)[2:end]]./3600

# print to latex the equivalent dataframe for time
tex_df_computational_time = latexify(df_computational_time; env=:table)

L"\begin{tabular}{cccccc}
$shapley_{enum}$ & $nucleolus_{enum}$ & $varcore_{enum}$ & $varleastcore_{enum}$ & $varcore_{iter}$ & $varleastcore_{iter}$\\
$0.33$ & $0.34$ & $0.34$ & $0.34$ & $0.92$ & $1.24$\\
\end{tabular}
"

In [50]:
# save latex code
open("latex_output.txt","w") do io
    println("Reward distribution table\n\n\n")
    print(io, tex_df_reward)
    println("Computational time table\n\n\n")
    print(io, tex_df_computational_time)
end

Reward distribution table



Computational time table



